# Finding histograms in a list

In a common case one defines histograms where the name incorporates settings i.e. a cut string and a tag. As histograms for cuts are typically in different lists the hisogram corresponding to a certain cut is unique. The goal is to write a method that returns the histogram with a certain tag in a general way. Assume the histograms are stored in a TObjArray, and there is only one TObjArray per file.

In [ ]:
TH1 *findHist(const char *filename, const char *histtag) {
    TH1 *hist = nullptr;
    std::unique_ptr<TFile> reader(TFile::Open(filename, "READ"));
    if(!reader.get() || reader->IsZombie()){
        std::cerr << "Failed accessing file " << filename << std::endl;
        return nullptr;
    }
    reader->ls();
    // your code here ...
    // Objects are in the TFile as directory, to be converted to a std::vector
    // TRangeDynCast automatically cast the object in ROOT containers to the inheriting type
    std::vector<TObject *> objects;
    for(auto o : TRangeDynCast<TKey>(gDirectory->GetListOfKeys())) objects.push_back(o->ReadObj());
    auto comp = [histtag] (TObject *o) { 
        // the comparator: capturing histtag by value
        std::cout << "Next object:   " << o->GetName() << std::endl; 
        std::cout << "Searching tag: " << histtag << std::endl; 
        // The actual find operation
        // return statement automatically determines return type
        return std::string(o->GetName()).find(histtag) != std::string::npos; 
    };  
    auto hptr = std::find_if(objects.begin(), objects.end(), comp);  // returning iterator to object
    if(hptr != objects.end()) {
        std::cout << "histo found" << std::endl;
        hist = static_cast<TH1 *>(*hptr);
    }
        
    // end your code
    if(hist) hist->SetDirectory(nullptr);    // Remove histogram from ROOT's file I/O
    return hist;
};

Test the method with a test file finding the histogram hPt.

In [ ]:
auto hist = findHist("./data/10111000101_trackqa.root", "hpt");
if(hist) {
    auto plot = new TCanvas("hPlot", "Plot", 800, 600);
    plot->cd();
    hist->Draw();    
    plot->Draw();
} else {
    std::cerr << "Histogram not found" << std::endl;
}
